In [ ]:
# Extract Feature Set (Select Particular Columns) from entire dataset

In [ ]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
import re


#### TODO: Specifiy Data Path Based on Participant/Your Computer

In [ ]:
# TODO: Specify Data Path based on participant/your computer
# Warning: this will probably take some time

file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/Preprocessed/p3_data_preprocessed2.csv'
data = pd.read_csv(file)

In [ ]:
data.head()

In [ ]:
# Drop Irrelevant Columns

# transcriptions
data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

data = data.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
data = data.drop(columns=['difficulty', 'skill', 'activity', 'of_face_id'])

In [ ]:
# Open-Face/Open-Pose Columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

only_ofop = []
performance = []
for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op_' in i or 'of_' in i:
        only_ofop.append(i)
    else:
        performance.append(i)
        
only_ofop.remove('of_timestamp')

In [ ]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

In [ ]:
# Constructed Feature: Approximated Distance to Camera!! 
# Notes Here: https://docs.google.com/document/d/1q5ZSRo8NgItPHMHeVJkU-_e6YFZgjRdni2M6l5zrJlc/edit?usp=sharing

data['of_gaze_distance'] = 0.0

for i, r in data.iterrows():    
    if (r['of_success'] == 0):
        data.at[i, 'of_gaze_distance'] = np.nan
    else:
        data.at[i, 'of_gaze_distance'] = (math.tan(r['of_gaze_angle_x'])*r['of_pose_Tz']) + r['of_pose_Tx']

# normalize it afterwards
#min_val = data['of_gaze_distance'].min()
#max_val = data['of_gaze_distance'].max()
#range_val = max_val - min_val

#data['of_gaze_distance'] = data['of_gaze_distance'].fillna(value=max_val)
#data['of_gaze_distance'] = (data['of_gaze_distance'] - min_val) / range_val

### Set Up Feature Set
#### (FS4: Performance + Handpicked OF/OP from Caitlyn's Intuition)
Note: keep timestamp and session_num until right before running the model in all feature sets

In [ ]:
# Feature Set 4: Performance + Handpicked OF/OP

hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y', 'gaze_distance']
for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked.append('op_Number of People')

for i in performance:
    hand_picked.append(i)

FS = data[hand_picked]

In [ ]:
test = FS.loc[FS['engagement'] == 0]
print(np.mean(test['of_gaze_distance']))

test = FS.loc[FS['engagement'] == 1]
print(np.mean(test['of_gaze_distance']))

### Write to File

In [ ]:
path_data = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p102_data_FS4new.csv'

FS.to_csv(path_data, index=False)

### Other Feature Sets

1. All OF/OP + Performance
2. Performance ONLY
3. All OF/OP ONLY
4. Performance + Handpicked OF/OP (from Caitlyn's intuition)
5. Performance + Selected OF/OP (from Feature Analysis) 

Note: keep timestamp and session_num until right before running the model in all feature sets

In [ ]:
# Feature Set 1: All OF/OP + Performance

# FS1 = data

In [ ]:
# Feature Set 2: Performance ONLY

# print(performance)

# FS2 = data[performance]

In [ ]:
# Feature Set 3: OF/OP ONLY  

# only_ofop.append('timestamp')
# only_ofop.append('session_num')
# only_ofop.append('engagement')

# FS3 = data[only_ofop]

In [ ]:
# Feature Set 4: Performance + Handpicked OF/OP

# hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
# for i,c in enumerate(hand_picked):
#     hand_picked[i] = 'of_' + c
# hand_picked.append('op_Number of People')

# for i in performance:
#     hand_picked.append(i)

# FS4 = data[hand_picked]

In [ ]:
# Feature Set 5: Performance + Selected OF/OP

# selected_ofop = ['of_AU04_c',   'of_AU06_c',    'of_AU07_c',    'of_AU14_c',    'of_AU15_c',    'of_AU25_c',    'of_confidence',    'of_frame',    'of_success',    'of_timestamp', 'op_p1_face_c25',    'op_p1_face_c43',    'op_p1_face_c44',    'op_p1_face_c45',    'op_p1_face_c46',    'op_p1_face_y7',    'op_p1_face_y8',    'op_p1_pose_c0',    'op_p1_pose_c15',    'op_p1_pose_c16']

# for i in performance:
#     selected_ofop.append(i)

# FS5 = data[selected_ofop]